# Documentation du script de training de recommandations produits en cas de rupture de stock 

## 1. Objectifs du script

Ce script a pour but de :
- Construire un dataset de substitution de produits (transactions, produits, substitutions).
- Effectuer du feature engineering pour générer des variables pertinentes pour prédire l’acceptation d’une substitution.
- Préparer les données via un pipeline scikit-learn pour la normalisation et l’encodage.
- Tester des modèles LGBMClassifier (classification binaire) et LGBMRanker (ranking) avec Hyperopt pour l’optimisation des hyperparamètres.
- Enregistrer automatiquement tous les résultats, métriques et modèles dans MLflow.
- Implémenter early stopping et pruning pour arrêter les modèles non performants rapidement.

In [3]:
print("""
Pipeline Recommandation Produits de Substitution
================================================

1️⃣ Chargement des données
   - substitutions.csv
   - produits.csv
   - transactions.csv

2️⃣ Construction du dataset + feature engineering
   [Transactions] --merge--> [Substitutions] --merge--> [Produits]
        |                                   |
        |                                   v
        |                             [Features Original / Subst]
        v
   [Label binaire: estAcceptee_bin]
   [Features supplémentaires: DiffPrix, MemeMarque, MemeNutriscore, MemeBio, Month, Day_of_week_name, ...]

3️⃣ Preprocessing
   - Numérique: Imputer (median) + StandardScaler
   - Catégoriel: Imputer (most_frequent) + OneHotEncoder
   - Combine via ColumnTransformer

4️⃣ Split train / val (temporel)
   - X_train, y_train
   - X_val, y_val
   - group_train, group_val (pour LGBMRanker)

5️⃣ Hyperopt + MLflow
   - Définition des espaces de recherche (num_leaves, learning_rate, n_estimators, etc.)
   - Objectif:
       - LGBMClassifier: maximiser AUC
       - LGBMRanker: maximiser NDCG@3
   - Early stopping natif LightGBM
   - Logging dans MLflow:
       - params
       - metrics (AUC, NDCG, hit_rate, etc.)
       - modèle
   - Pruning simple:
       - classifier: abandon si auc < 0.55
       - ranker: abandon si ndcg@3 < 0.05

6️⃣ Résultat
   - Best hyperparameters
   - Best model enregistré pour production
   - Comparaison automatique via MLflow UI

7️⃣ Production
   - Pipeline complet prêt à servir pour la recommandation de produits de substitution
""")



Pipeline Recommandation Produits de Substitution

1️⃣ Chargement des données
   - substitutions.csv
   - produits.csv
   - transactions.csv

2️⃣ Construction du dataset + feature engineering
   [Transactions] --merge--> [Substitutions] --merge--> [Produits]
        |                                   |
        |                                   v
        |                             [Features Original / Subst]
        v
   [Label binaire: estAcceptee_bin]
   [Features supplémentaires: DiffPrix, MemeMarque, MemeNutriscore, MemeBio, Month, Day_of_week_name, ...]

3️⃣ Preprocessing
   - Numérique: Imputer (median) + StandardScaler
   - Catégoriel: Imputer (most_frequent) + OneHotEncoder
   - Combine via ColumnTransformer

4️⃣ Split train / val (temporel)
   - X_train, y_train
   - X_val, y_val
   - group_train, group_val (pour LGBMRanker)

5️⃣ Hyperopt + MLflow
   - Définition des espaces de recherche (num_leaves, learning_rate, n_estimators, etc.)
   - Objectif:
       - LGBMClassifie

#### A. Import et préparation de l'environnement

In [2]:
import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import (
    roc_auc_score, average_precision_score, log_loss,
    precision_score, recall_score, f1_score
)

import mlflow
import mlflow.lightgbm
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from lightgbm import LGBMClassifier, LGBMRanker, early_stopping, log_evaluation

from pathlib import Path
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from catboost import CatBoostClassifier


/home/eric/.cache/pypoetry/virtualenvs/algo-reco-Zao1WwXe-py3.12/lib/python3.12/site-packages/hyperopt/atpe.py:19: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


#### B. Détection de la racine du projet et chemins relatifs

In [15]:
def find_project_root(marker=".git"):
    path = Path().resolve()
    while path != path.parent:
        if (path / marker).exists():
            return path
        path = path.parent
    raise FileNotFoundError(f"Project root with {marker} not found")
ROOT_DIR = find_project_root()
DATA_RAW_DIR = ROOT_DIR / "data" / "raw"
MLFLOW_TRACKING_URI = "http://localhost:5555"
EXP_NAME = "stockout_substitution_hyperopt_classifier_ranker_6"


#### C. Chargement des données

In [4]:
#TODO connecter directement via utils de GCP
substitutions = pd.read_csv(DATA_RAW_DIR / "substitutions" / "raw_substitutions_substitutions.csv")
produits = pd.read_csv(DATA_RAW_DIR / "produits" / "raw_produits_produits.csv")
transactions = pd.read_csv(DATA_RAW_DIR / "transactions" / "raw_transactions_transactions.csv")

#### D. Construction du dataset et feature engineering

Explications :
- On fait un merge pour récupérer toutes les informations sur le produit original et la substitution.
- Création de features binaires et numériques pertinentes :
- MemeMarque, MemeNutriscore, DiffPrix, etc.
- Extraction du mois et jour de la semaine depuis la date.

In [5]:
def merge_and_add_suffix(df_add_suffix, df_keep, suffix, column_to_merge):
    df_add = df_add_suffix.copy()
    df_add = df_add.rename(columns={c: c + suffix for c in df_add.columns if c != column_to_merge})
    return pd.merge(df_keep, df_add, left_on=column_to_merge, right_on=column_to_merge, how='left')

def build_dataset(transactions, substitutions, produits):
    subs_prod_orig = merge_and_add_suffix(produits, substitutions, 'Original', 'idProduitOriginal')
    subs_prod_orig_subst = merge_and_add_suffix(produits, subs_prod_orig, 'Substitution', 'idProduitSubstitution')
    df = pd.merge(transactions, subs_prod_orig_subst,
                  left_on=['idProduit','idTransaction'],
                  right_on=['idProduitOriginal','idTransaction'], how='inner')
    df['estAcceptee_bin'] = (~df['estAcceptee']).astype(int)
    df['date'] = pd.to_datetime(df['dateHeureTransaction'])
    df['Month'] = df['date'].dt.month
    df['Day_of_week_name'] = df['date'].dt.day_name()
    df["MemeMarque"] = (df["marqueOriginal"] == df["marqueSubstitution"]).astype(int)
    df["MemeNutriscore"] = (df["nutriscoreOriginal"] == df["nutriscoreSubstitution"]).astype(int)
    df["MemeConditionnement"] = (df["conditionnementOriginal"] == df["conditionnementSubstitution"]).astype(int)
    df["MemeTypeMarque"] = (df["typeMarqueOriginal"] == df["typeMarqueSubstitution"]).astype(int)
    df["DiffPrix"] = df["prixSubstitution"] - df["prixOriginal"]
    df["MemeBio"] = ((df["estBioOriginal"] == True) & (df["estBioSubstitution"] == True)).astype(int)
    return df

df = build_dataset(transactions, substitutions, produits)


#### E. Preprocessing

Explications:
- Standardisation et imputation pour les variables numériques.
- One-Hot Encoding et imputation pour les variables catégorielles.
- ColumnTransformer combine les deux types de transformations.

In [6]:
features_num = ["DiffPrix", "MemeMarque", "MemeNutriscore", "MemeBio",
                "prixOriginal", "MemeConditionnement", "MemeTypeMarque", "estBioOriginal", "Month"]
features_cat = ["categorieOriginal", "marqueOriginal", "typeMarqueOriginal", "nutriscoreOriginal",
                "origineOriginal", "conditionnementOriginal", "categorieSubstitution",
                "typeMarqueSubstitution", "origineSubstitution", "Day_of_week_name"]

X = df[features_num + features_cat]
y = df["estAcceptee_bin"]

# Split temporel
cutoff_idx = int(len(df) * 0.8)
X_train_raw, X_val_raw = X.iloc[:cutoff_idx], X.iloc[cutoff_idx:]
y_train, y_val = y.iloc[:cutoff_idx], y.iloc[cutoff_idx:]

# Pipeline
numeric_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])
categorical_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown='ignore'))
])
preprocessor = ColumnTransformer([
    ("num", numeric_transformer, features_num),
    ("cat", categorical_transformer, features_cat)
])

X_train = preprocessor.fit_transform(X_train_raw)
X_val = preprocessor.transform(X_val_raw)

#### F. Gestion des groupes pour LGBMRanker

Explications:
- LGBMRanker nécessite un vecteur indiquant le nombre de produits par stockout event.
- Chaque idTransaction représente un groupe.

In [7]:
group_train = df.iloc[:cutoff_idx].groupby('idTransaction').size().to_numpy()
group_val = df.iloc[cutoff_idx:].groupby('idTransaction').size().to_numpy()

#### G. Espace de recherche Hyperopt (space_lgbm_ranker)

Explications:
- Définit les hyperparamètres que Hyperopt va explorer pour optimiser le LGBMRanker.
- num_leaves, n_estimators, min_child_samples sont des entiers (d’où la conversion plus tard).
- learning_rate est sur une échelle logarithmique pour couvrir efficacement les petits et grands taux d’apprentissage.
- subsample et colsample_bytree contrôlent la variance du modèle et la régularisation.

In [8]:
space_lgbm_ranker = {
    "num_leaves": hp.quniform("num_leaves", 31, 127, 1),
    "learning_rate": hp.loguniform("learning_rate", np.log(0.01), np.log(0.2)),
    "n_estimators": hp.quniform("n_estimators", 300, 1200, 50),
    "min_child_samples": hp.quniform("min_child_samples", 20, 100, 5),
    "subsample": hp.uniform("subsample", 0.7, 1.0),
    "colsample_bytree": hp.uniform("colsample_bytree", 0.7, 1.0),
}

#### H. Fonction de calcul des métriques de ranking (compute_ranking_metrics)

Explications:
- group : vecteur indiquant la taille de chaque transaction / événement. Pour chaque groupe :
    - Trier les prédictions par score décroissant.
    - Calculer ndcg@k et hit_rate@k pour k = 1, 3, 5.

ndcg@k (Normalized Discounted Cumulative Gain): mesure la qualité du ranking en pondérant les positions des items pertinents.
hit_rate@k: indique si au moins un produit pertinent apparaît dans le top k.

=> Retourne un dictionnaire avec toutes les métriques moyennes sur tous les groupes.

In [9]:
def compute_ranking_metrics(y_true, y_score, group, ks=(1,3,5)):
    metrics = {}
    idx = 0
    per_k = {k: {"ndcg": [], "hit": []} for k in ks}
    for g in group:
        y_g = np.asarray(y_true[idx: idx + g])
        s_g = np.asarray(y_score[idx: idx + g])
        idx += g
        order = np.argsort(-s_g)
        rels_sorted = y_g[order]
        for k in ks:
            rels_k = rels_sorted[:k]
            per_k[k]["ndcg"].append(np.sum(rels_k / np.log2(np.arange(2, len(rels_k)+2))))
            per_k[k]["hit"].append(float(np.any(rels_k > 0)))
    for k in ks:
        metrics[f"ndcg_at_{k}"] = float(np.mean(per_k[k]["ndcg"]))
        metrics[f"hit_rate_at_{k}"] = float(np.mean(per_k[k]["hit"]))

    return metrics


In [18]:
def compute_classification_metrics(y_true, y_proba, threshold=0.5):
    y_pred = (y_proba >= threshold).astype(int)
    return {
        "auc": float(roc_auc_score(y_true, y_proba)) if len(np.unique(y_true)) > 1 else np.nan,
        "pr_auc": float(average_precision_score(y_true, y_proba)),
        "logloss": float(log_loss(y_true, y_proba)),
        "precision": float(precision_score(y_true, y_pred, zero_division=0)),
        "recall": float(recall_score(y_true, y_pred, zero_division=0)),
        "f1": float(f1_score(y_true, y_pred, zero_division=0)),
    }

#### Dictionnaires des modèles

In [10]:
models = {
    "LogReg": {"model_class": LogisticRegression, "param_grid": {"C": [0.1,1.0,10.0], "penalty":["l2"]}, "fixed_params": {"solver":"lbfgs","max_iter":2000,"n_jobs":-1,"random_state":42}, "type":"classification"},
    "XGBClassifier": {"model_class": XGBClassifier, "param_grid": {"n_estimators":[500,1000],"max_depth":[4,6,8],"learning_rate":[0.03,0.05,0.1],"subsample":[0.7,0.9,1.0],"colsample_bytree":[0.7,0.9,1.0],"min_child_weight":[1,5,10],"reg_alpha":[0.0,0.1,1.0],"reg_lambda":[1.0,2.0,5.0]}, "fixed_params":{"objective":"binary:logistic","eval_metric":"auc","tree_method":"hist","random_state":42,"n_jobs":-1}, "type":"classification"},
    "LGBMClassifier": {"model_class": LGBMClassifier, "param_grid":{"num_leaves":[31,63,127],"learning_rate":[0.03,0.05,0.1],"n_estimators":[500,1000],"min_child_samples":[20,50,100],"subsample":[0.7,0.9,1.0],"colsample_bytree":[0.7,0.9,1.0],"reg_alpha":[0.0,0.1,1.0],"reg_lambda":[0.0,0.1,1.0]}, "fixed_params":{"objective":"binary","metric":"auc","random_state":42,"n_jobs":-1}, "type":"classification"},
    "CatBoostClassifier": {"model_class": CatBoostClassifier, "param_grid":{"depth":[6,8,10],"learning_rate":[0.03,0.05,0.1],"iterations":[500,1000],"l2_leaf_reg":[1,3,5,9],"subsample":[0.7,0.9,1.0],"rsm":[0.7,0.9,1.0]}, "fixed_params":{"loss_function":"Logloss","eval_metric":"AUC","random_seed":42,"verbose":0}, "type":"classification"},
    "LGBMRanker": {"model_class": LGBMRanker, "param_grid":{"num_leaves":[31,63,127],"learning_rate":[0.03,0.05,0.1],"n_estimators":[500,1000],"min_child_samples":[20,50,100],"subsample":[0.7,0.9,1.0],"colsample_bytree":[0.7,0.9,1.0]}, "fixed_params":{"objective":"lambdarank","metric":"ndcg","random_state":42,"n_jobs":-1}, "type":"ranking"}
}

#### I. Fonction objectif Hyperopt (objective_lgbm_ranker)

Explications:
- **Conversion des hyperparamètres** en entiers si nécessaire (num_leaves, n_estimators, min_child_samples).
- **MLflow** : chaque appel à la fonction ouvre un run nested pour enregistrer paramètres et métriques.
- LGBMRanker.fit:
        - group_train : indique la taille des groupes (transactions).
        - eval_set et eval_group pour monitoring sur validation.
        - callbacks : early stopping + logging sans verbosité.

- **Métriques** : calcul via compute_ranking_metrics.
- **Pruning** : si ndcg@3 < 0.05, on abandonne ce modèle pour économiser du temps.
- **La loss** retournée est négative pour que Hyperopt maximise ndcg@3.

In [11]:
def safe_log_metrics(metrics_dict):
    safe_metrics = {}
    for k, v in metrics_dict.items():
        try:
            safe_metrics[k] = float(v)
        except (TypeError, ValueError):
            safe_metrics[k] = np.nan
    mlflow.log_metrics(safe_metrics)


In [12]:
def objective_ranker(params, X_train, y_train, X_val, y_val, group_train, group_val, model_name):
    params = {k:int(v) if k in ["num_leaves","n_estimators","min_child_samples"] else v for k,v in params.items()}
    
    with mlflow.start_run(nested=True):
        mlflow.set_tag("model_name", model_name)
        mlflow.set_tag("model_type", "ranking")

        model = LGBMRanker(**params)
        model.fit(
            X_train, y_train,
            group=group_train,
            eval_set=[(X_val, y_val)],
            eval_group=[group_val],
            eval_metric="ndcg",
            callbacks=[early_stopping(stopping_rounds=50, verbose=False),
                       log_evaluation(period=0)]
        )
        best_iter = model.best_iteration_
        mlflow.log_metric("best_iteration", best_iter)

        scores = model.predict(X_val, num_iteration=best_iter)
        metrics = compute_ranking_metrics(y_val, scores, group_val)

        mlflow.log_params(params)
        safe_log_metrics(metrics)
        mlflow.lightgbm.log_model(model, "model")
        
        return {"loss": -metrics["ndcg_at_3"], "status": STATUS_OK}


In [13]:
def objective_classifier(params, model_class, X_train, y_train, X_val, y_val, model_name):
    params = {k:int(v) if isinstance(v,float) and v.is_integer() else v for k,v in params.items()}
    
    with mlflow.start_run(nested=True):
        mlflow.set_tag("model_name", model_name)
        mlflow.set_tag("model_type", "classification")

        model = model_class(**params)
        model.fit(X_train, y_train)

        y_proba = model.predict_proba(X_val)[:,1]
        metrics = compute_classification_metrics(y_val, y_proba)

        mlflow.log_params(params)
        safe_log_metrics(metrics)

        if isinstance(model,LGBMClassifier):
            mlflow.lightgbm.log_model(model,"model")

        return {"loss": -metrics["auc"], "status": STATUS_OK}

In [16]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)  # ou localhost si accessible
try:
    mlflow.create_experiment(EXP_NAME)
except mlflow.exceptions.MlflowException:
    pass
mlflow.set_experiment(EXP_NAME)

<Experiment: artifact_location='mlflow-artifacts:/7', creation_time=1767777786934, experiment_id='7', last_update_time=1767777786934, lifecycle_stage='active', name='stockout_substitution_hyperopt_classifier_ranker_6', tags={}>

#### J. Lancement de l'optimisation Hyperopt

Explications :
- fmin : trouve les meilleurs hyperparamètres pour maximiser ndcg@3.
- algo=tpe.suggest : utilise **l’algorithme Tree-structured Parzen Estimator** pour l’optimisation bayésienne.
- max_evals=50 : nombre d’itérations Hyperopt.
- trials_ranker : enregistre tous les essais, métriques et hyperparamètres.
- rstate : garantit la **reproductibilité.**

In [19]:
for name, cfg in models.items():
    print(f"=== Optimisation {name} ===")
    
    # Création de l'espace Hyperopt
    space = {k: hp.choice(k, v) if isinstance(v, list) else v for k, v in cfg["param_grid"].items()}
    trials = Trials()
    
    if cfg["type"] == "classification":
        fmin(
            fn=lambda p: objective_classifier(
                p,
                cfg["model_class"],
                X_train,
                y_train,
                X_val,
                y_val,
                model_name=name
            ),
            space=space,
            algo=tpe.suggest,
            max_evals=20,
            trials=trials,
            rstate=np.random.default_rng(42)
        )
    else:
        fmin(
            fn=lambda p: objective_ranker(
                p,
                X_train,
                y_train,
                X_val,
                y_val,
                group_train,
                group_val,
                model_name=name
            ),
            space=space,
            algo=tpe.suggest,
            max_evals=20,
            trials=trials,
            rstate=np.random.default_rng(42)
        )


=== Optimisation LogReg ===
  0%|          | 0/20 [00:00<?, ?trial/s, best loss=?]

/home/eric/.cache/pypoetry/virtualenvs/algo-reco-Zao1WwXe-py3.12/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1135: FutureWarning: 'penalty' was deprecated in version 1.8 and will be removed in 1.10. To avoid this warning, leave 'penalty' set to its default value and use 'l1_ratio' or 'C' instead. Use l1_ratio=0 instead of penalty='l2', l1_ratio=1 instead of penalty='l1', and C=np.inf instead of penalty=None.
  warnings.warn(



🏃 View run magnificent-gnu-774 at: http://localhost:5555/#/experiments/7/runs/207c4f55a6514de6995aa7388639bc67

🧪 View experiment at: http://localhost:5555/#/experiments/7

  5%|▌         | 1/20 [00:00<00:10,  1.82trial/s, best loss: -0.6990719003179975]

/home/eric/.cache/pypoetry/virtualenvs/algo-reco-Zao1WwXe-py3.12/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1135: FutureWarning: 'penalty' was deprecated in version 1.8 and will be removed in 1.10. To avoid this warning, leave 'penalty' set to its default value and use 'l1_ratio' or 'C' instead. Use l1_ratio=0 instead of penalty='l2', l1_ratio=1 instead of penalty='l1', and C=np.inf instead of penalty=None.
  warnings.warn(



🏃 View run nosy-squid-670 at: http://localhost:5555/#/experiments/7/runs/604cb181601f4044827c9051f2a5fcc3

🧪 View experiment at: http://localhost:5555/#/experiments/7                     

 10%|█         | 2/20 [00:01<00:15,  1.16trial/s, best loss: -0.6990719003179975]

...

🏃 View run secretive-ape-612 at: http://localhost:5555/#/experiments/7/runs/1d55f3ce4a104f3cbaa7716234813ae2

🧪 View experiment at: http://localhost:5555/#/experiments/7                      

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002834 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 826                                                  
[LightGBM] [Info] Number of data points in the train set: 90941, number of used features: 154
 90%|█████████ | 18/20 [01:50<00:10,  5.06s/trial, best loss: -0.8573883922378597]

/home/eric/.cache/pypoetry/virtualenvs/algo-reco-Zao1WwXe-py3.12/lib/python3.12/site-packages/sklearn/utils/validation.py:2691: UserWarning: X does not have valid feature names, but LGBMRanker was fitted with feature names
  warnings.warn(

2026/01/07 09:34:47 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



🏃 View run honorable-smelt-850 at: http://localhost:5555/#/experiments/7/runs/5910e6f65ad645ffb5716d529b2608d4

🧪 View experiment at: http://localhost:5555/#/experiments/7                      

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002779 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 826                                                  
[LightGBM] [Info] Number of data points in the train set: 90941, number of used features: 154
 95%|█████████▌| 19/20 [01:53<00:04,  4.53s/trial, best loss: -0.8573883922378597]

/home/eric/.cache/pypoetry/virtualenvs/algo-reco-Zao1WwXe-py3.12/lib/python3.12/site-packages/sklearn/utils/validation.py:2691: UserWarning: X does not have valid feature names, but LGBMRanker was fitted with feature names
  warnings.warn(

2026/01/07 09:34:50 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



🏃 View run valuable-conch-790 at: http://localhost:5555/#/experiments/7/runs/46ad7c66a9484c7a84cf379747c88287

🧪 View experiment at: http://localhost:5555/#/experiments/7                      

100%|██████████| 20/20 [01:56<00:00,  5.83s/trial, best loss: -0.8573883922378597]


In [ ]:
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier, LGBMRanker
from catboost import CatBoostClassifier

# -------------------------------------------------------------------
# Pourquoi tester ces algorithmes pour la recommandation de produits
# en cas de rupture de stock ?
#
# L’objectif est de prédire ou de classer les meilleurs produits de
# substitution à proposer à un client lorsqu’un article est indisponible.
# Nous testons plusieurs familles de modèles afin de comparer :
#   - la capacité de généralisation,
#   - la performance en classification et en ranking,
#   - la robustesse face à des features hétérogènes (prix, catégorie,
#     similarité, contexte client, etc.).
#
# 1) Logistic Regression
#    - Sert de modèle baseline simple et interprétable.
#    - Permet de vérifier que les features contiennent bien un signal
#      prédictif (sanity check).
#    - Facilite l’analyse des coefficients et la compréhension métier.
#
# 2) XGBClassifier (XGBoost)
#    - Modèle de gradient boosting très performant sur données tabulaires.
#    - Capable de capturer des relations non linéaires complexes entre
#      produits et contexte client.
#    - Sert de référence "haut niveau" en classification supervisée.
#
# 3) LGBMClassifier (LightGBM)
#    - Alternative plus rapide et scalable au XGBoost.
#    - Très efficace sur de grands volumes de données et de nombreuses
#      features.
#    - Utilisé ici pour prédire la probabilité qu’un produit substitut
#      soit accepté par le client.
#
# 4) LGBMRanker (LightGBM - Learning to Rank)
#    - Modèle spécifiquement conçu pour les problèmes de ranking.
#    - Permet de classer plusieurs produits candidats pour une même
#      rupture de stock et de sélectionner le meilleur (Top-1 ou Top-K).
#    - Particulièrement adapté aux systèmes de recommandation.
#
# 5) CatBoostClassifier
#    - Modèle de boosting optimisé pour les variables catégorielles.
#    - Réduit le besoin de preprocessing (encodage) des catégories.
#    - Souvent très performant dans les contextes e-commerce avec
#      catégories, marques et attributs produits.
#
# Cette approche multi-modèles permet d’identifier le meilleur compromis
# entre performance, interprétabilité et robustesse pour le moteur de
# recommandation de produits de substitution.
# -------------------------------------------------------------------


# Définir les modèles et leurs grilles de paramètres
models = {

    # =========================
    # Logistic Regression (Baseline)
    # =========================
    "LogReg": {
        "model_class": LogisticRegression,
        "param_grid": {
            "C": [0.1, 1.0, 10.0],
            "penalty": ["l2"],
        },
        "fixed_params": {
            "solver": "lbfgs",
            "max_iter": 2000,
            "n_jobs": -1,
            "random_state": 42,
        },
    },

    # =========================
    # XGBoost Classifier
    # =========================
    "XGBClassifier": {
        "model_class": XGBClassifier,
        "param_grid": {
            "n_estimators": [500, 1000],
            "max_depth": [4, 6, 8],
            "learning_rate": [0.03, 0.05, 0.1],
            "subsample": [0.7, 0.9, 1.0],
            "colsample_bytree": [0.7, 0.9, 1.0],
            "min_child_weight": [1, 5, 10],
            "reg_alpha": [0.0, 0.1, 1.0],
            "reg_lambda": [1.0, 2.0, 5.0],
        },
        "fixed_params": {
            "objective": "binary:logistic",
            "eval_metric": "auc",
            "tree_method": "hist",
            "random_state": 42,
            "n_jobs": -1,
        },
    },

    # =========================
    # LightGBM Classifier
    # =========================
    "LGBMClassifier": {
        "model_class": LGBMClassifier,
        "param_grid": {
            "num_leaves": [31, 63, 127],
            "learning_rate": [0.03, 0.05, 0.1],
            "n_estimators": [500, 1000],
            "min_child_samples": [20, 50, 100],
            "subsample": [0.7, 0.9, 1.0],
            "colsample_bytree": [0.7, 0.9, 1.0],
            "reg_alpha": [0.0, 0.1, 1.0],
            "reg_lambda": [0.0, 0.1, 1.0],
        },
        "fixed_params": {
            "objective": "binary",
            "metric": "auc",
            "random_state": 42,
            "n_jobs": -1,
        },
    },

    # =========================
    # LightGBM Ranker
    # =========================
    "LGBMRanker": {
        "model_class": LGBMRanker,
        "param_grid": {
            "objective": ["lambdarank"],
            "metric": ["ndcg"],
            "num_leaves": [31, 63, 127],
            "learning_rate": [0.03, 0.05, 0.1],
            "n_estimators": [500, 1000],
            "min_child_samples": [20, 50, 100],
            "subsample": [0.7, 0.9, 1.0],
            "colsample_bytree": [0.7, 0.9, 1.0],
        },
        "fixed_params": {
            "random_state": 42,
            "n_jobs": -1,
        },
    },

    # =========================
    # CatBoost Classifier
    # =========================
    "CatBoostClassifier": {
        "model_class": CatBoostClassifier,
        "param_grid": {
            "depth": [6, 8, 10],
            "learning_rate": [0.03, 0.05, 0.1],
            "iterations": [500, 1000],
            "l2_leaf_reg": [1, 3, 5, 9],
            "subsample": [0.7, 0.9, 1.0],
            "rsm": [0.7, 0.9, 1.0],
        },
        "fixed_params": {
            "loss_function": "Logloss",
            "eval_metric": "AUC",
            "random_seed": 42,
            "verbose": 0,
        },
    },
}



In [ ]:
import numpy as np
from sklearn.metrics import (
    roc_auc_score, average_precision_score, log_loss,
    precision_score, recall_score, f1_score
)
# Métriques ranking simples (binaire ou grades) 
def dcg_at_k(rels, k):
    rels = np.asarray(rels)[:k]
    if rels.size == 0:
        return 0.0
    discounts = 1.0 / np.log2(np.arange(2, rels.size + 2))
    return float(np.sum(rels * discounts))

def ndcg_at_k(rels, k):
    dcg = dcg_at_k(rels, k)
    ideal = dcg_at_k(sorted(rels, reverse=True), k)
    return 0.0 if ideal == 0 else float(dcg / ideal)

def mrr_at_k(rels, k):
    rels = np.asarray(rels)[:k]
    hits = np.where(rels > 0)[0]
    return 0.0 if hits.size == 0 else float(1.0 / (hits[0] + 1))

def hit_rate_at_k(rels, k):
    rels = np.asarray(rels)[:k]
    return float(np.any(rels > 0))

def precision_at_k(rels, k):
    rels = np.asarray(rels)[:k]
    return float(np.mean(rels > 0)) if rels.size else 0.0

def recall_at_k(rels, k):
    rels = np.asarray(rels)
    total_pos = np.sum(rels > 0)
    if total_pos == 0:
        return 0.0
    return float(np.sum(rels[:k] > 0) / total_pos)